<a href="https://colab.research.google.com/github/PrakharPatni08/Gen-AI/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##GAN Variants:
* DCGAN(deep convolutional GAN) - It is a type of GAN that is specially designed for generating realistic images by using deep convolutional NN instead of fully connected layer. It was introduced in 2015.
Used in fashion applications, face generation, data augumentation.

 How DCGAN works:

 Generator
 1. Random inputs to generator.
 2. Transposed conv 2D layer in generator.
 3. Batch normalization
 4. ReLU AF

 Discriminator
 1. Takes generated image as input.
 2. Use conv 2D layer.
 3. LeakyReLU AF.
 4. No pooling layers.

  WORKFLOW:

* Conditional GAN -

* CycleGAN


In [ ]:
# ------------------------- Importing Required Libraries ------------------------- #
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

# ------------------------- Configuration / Hyperparams ------------------------- #
BUFFER_SIZE = 60000
BATCH_SIZE = 256
IMG_HEIGHT = 28
IMG_WIDTH = 28
CHANNELS = 1
EPOCHS = 100
LATENT_DIM = 100
NUM_EXAMPLES_TO_GENERATE = 16
SAVE_INTERVAL = 5    # save generated images every n epochs

OUTPUT_DIR = "generated_images"
CHECKPOINT_DIR = "checkpoints"

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Set inputs for consistent image generation
inputs = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, LATENT_DIM])

In [ ]:
# ------------------------- Load and preprocess dataset ------------------------- #
def load_fashion_mnist():
    (train_images, _), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
    # expand dims to (N,28,28,1)
    train_images = train_images.reshape((-1, IMG_HEIGHT, IMG_WIDTH, CHANNELS)).astype('float32')
    # scale to [-1, 1] for tanh output
    train_images = (train_images - 127.5) / 127.5
    return train_images

train_images = load_fashion_mnist()

train_dataset = tf.data.Dataset.from_tensor_slices(train_images) \
    .shuffle(BUFFER_SIZE) \
    .batch(BATCH_SIZE, drop_remainder=True) \
    .prefetch(tf.data.AUTOTUNE)

In [ ]:

# ------------------------- Build the Generator ------------------------- #
def make_generator_model():
    model = tf.keras.Sequential(name="Generator")
    # Dense -> reshape
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(LATENT_DIM,)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Reshape((7, 7, 256)))  # 7x7x256

    # Upsample to 14x14
    model.add(layers.Conv2DTranspose(128, kernel_size=5, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # Upsample to 28x28
    model.add(layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # Final conv to 1 channel
    model.add(layers.Conv2DTranspose(CHANNELS, kernel_size=5, strides=1, padding='same', use_bias=False, activation='tanh'))
    return model

generator = make_generator_model()
generator.summary()


Model: "Generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 12544)          │     1,254,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 12544)          │        50,176 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 14, 14, 128)    │       819,200 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_4 (ReLU)                  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_4              │ (None, 28, 28, 64)     │       204,800 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_5 (ReLU)                  │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_5              │ (None, 28, 28, 1)      │         1,600 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,330,944 (8.89 MB)

 Trainable params: 2,305,472 (8.79 MB)

 Non-trainable params: 25,472 (99.50 KB)

In [ ]:
# ------------------------- Build the Discriminator ------------------------- #
def make_discriminator_model():
    model = tf.keras.Sequential(name="Discriminator")
    model.add(layers.Conv2D(64, kernel_size=5, strides=2, padding='same',
                            input_shape=[IMG_HEIGHT, IMG_WIDTH, CHANNELS]))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, kernel_size=5, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

discriminator = make_discriminator_model()
discriminator.summary()

Model: "Discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 14, 14, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 7, 7, 128)      │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │         6,273 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 212,865 (831.50 KB)

 Trainable params: 212,865 (831.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ------------------------- Loss and optimizers ------------------------- #
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    # real labels = 1, fake labels = 0
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    # want discriminator to predict 1 for generated images
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
# ------------------------- Checkpoints ------------------------- #
checkpoint_prefix = os.path.join(CHECKPOINT_DIR, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
manager = tf.train.CheckpointManager(checkpoint, CHECKPOINT_DIR, max_to_keep=3)

In [ ]:
# ------------------------- generate & save images ------------------------- #
def generate_and_save_images(model, epoch, test_input, grid_size=(4,4)):
    """
    model: generator
    epoch: int
    test_input: latent vectors (NUM_EXAMPLES_TO_GENERATE, LATENT_DIM)
    """
    predictions = model(test_input, training=False)  # shape: (N, 28, 28, 1)
    # Rescale from [-1,1] to [0,1]
    preds = (predictions + 1.0) / 2.0
    preds = preds.numpy()

    fig = plt.figure(figsize=(grid_size[1]*2, grid_size[0]*2))
    for i in range(preds.shape[0]):
        plt.subplot(grid_size[0], grid_size[1], i+1)
        image = preds[i, :, :, 0]
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.suptitle(f"Epoch {epoch}", fontsize=12)
    fname = os.path.join(OUTPUT_DIR, f"image_at_epoch_{epoch:04d}.png")
    plt.savefig(fname, bbox_inches='tight')
    plt.close(fig)

In [ ]:
# ------------------------- Training loop ------------------------- #
@tf.function
def train_step(images):
    inputs = tf.random.normal([BATCH_SIZE, LATENT_DIM])

    # Train discriminator
    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape:
        generated_images = generator(inputs, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        disc_loss = discriminator_loss(real_output, fake_output)
        gen_loss = generator_loss(fake_output)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)

    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

    return gen_loss, disc_loss

def train(dataset, epochs):
    start = time.time()
    for epoch in range(1, epochs + 1):
        epoch_gen_loss = 0.0
        epoch_disc_loss = 0.0
        steps = 0
        for image_batch in dataset:
            g_loss, d_loss = train_step(image_batch)
            epoch_gen_loss += g_loss.numpy()
            epoch_disc_loss += d_loss.numpy()
            steps += 1

        epoch_gen_loss /= steps
        epoch_disc_loss /= steps

        # produce images for monitoring
        if (epoch % SAVE_INTERVAL == 0) or (epoch == 1) or (epoch == epochs):
            generate_and_save_images(generator, epoch, inputs)

        # Save checkpoint
        manager.save()

        print(f"Epoch {epoch}/{epochs}  Gen loss: {epoch_gen_loss:.4f}  Disc loss: {epoch_disc_loss:.4f}  Time elapsed: {time.time()-start:.1f}s")

    # Generate final images
    generate_and_save_images(generator, epochs, inputs)

# ------------------------- Entry point ------------------------- #
if __name__ == "__main__":
    print("Starting DCGAN training on Fashion-MNIST")
    print(f"Training for {EPOCHS} epochs, batch size {BATCH_SIZE}, latent dim {LATENT_DIM}")
    # Optional: restore latest checkpoint
    if manager.latest_checkpoint:
        print("Restoring from checkpoint:", manager.latest_checkpoint)
        checkpoint.restore(manager.latest_checkpoint)

    train(train_dataset, EPOCHS)
    print("Training finished. Generated images are in:", OUTPUT_DIR)
    print("Checkpoints in:", CHECKPOINT_DIR)

Starting DCGAN training on Fashion-MNIST
Training for 100 epochs, batch size 256, latent dim 100
Restoring from checkpoint: checkpoints/ckpt-1
Epoch 1/100  Gen loss: 1.7024  Disc loss: 0.8983  Time elapsed: 1007.8s
Epoch 2/100  Gen loss: 0.8503  Disc loss: 1.3241  Time elapsed: 2010.7s
Epoch 3/100  Gen loss: 0.9030  Disc loss: 1.2408  Time elapsed: 3016.8s
Epoch 4/100  Gen loss: 0.6938  Disc loss: 1.4408  Time elapsed: 4012.8s
Epoch 5/100  Gen loss: 0.7304  Disc loss: 1.3614  Time elapsed: 5001.1s
Epoch 6/100  Gen loss: 0.7161  Disc loss: 1.3479  Time elapsed: 5998.0s
Epoch 7/100  Gen loss: 0.7259  Disc loss: 1.3313  Time elapsed: 6997.3s
Epoch 8/100  Gen loss: 0.8565  Disc loss: 1.2718  Time elapsed: 8000.0s
Epoch 9/100  Gen loss: 0.6981  Disc loss: 1.3485  Time elapsed: 8995.5s
